In [ ]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [128]:
response = requests.get("https://topwebfiction.com/?ranking=at")
soup = BeautifulSoup(response.text, 'html.parser')

#FIND THE STORY TITLES ON ONE PAGE
story_titles = soup.findAll('span', attrs={"class":"title"})
titles = []
for row in story_titles:
    titles.append(row.text.strip())
    
#FIND THE STORY AUTHORS ON ONE PAGE
story_authors = soup.findAll('span', attrs={"class":"byline"})
authors = []
for row in story_authors:
    authors.append(row.text.strip())
for x in range(len(authors)):
    authors[x] = re.sub('by ','',authors[x])
    
#GET STORY VOTES FOR ONE PAGE
story_votes = soup.findAll('td', attrs={"class":"info"})
votes = []
for x in story_votes:
    votes.append(x.text)
votes2 = votes
for x in range(len(votes)):
    votes2[x] = re.sub(r'\n', '', str(votes[x]))
    votes2[x] = re.sub(r'\xa0boosters', '', str(votes2[x]))
    
#GET STORY TAGS FOR ONE PAGE
story_tags = soup.findAll('p', attrs={"class":"tags"})
tags = []
for x in story_tags:
    tags.append(x.text)
for x in range(len(tags)):
    tags[x] = re.sub(' ', '-', str(tags[x]))
    tags[x] = re.sub(r'\n', ' ', str(tags[x]))
    tags[x] = re.split(' ', str(tags[x]))
    
links = []
for link in soup.find_all('a', attrs={'href': re.compile("^https://topwebfiction.com/listings/")}):
    # display the actual urls
    links.append(link.get('href'))

def get_related_links_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    related_stories_soup = soup.findAll('span', attrs={"class":"title"})
    return(related_stories_soup)

def get_related_links(soup):
    link_list = []
    for x in range(len(soup)):
        link_list.append(soup[x].text)
    return(link_list[10:])

def relatedLinkFetch(url): 
    linksoup = get_related_links_soup(url)
    return(get_related_links(linksoup))

rel_stories = []
for x in links:
    rel_stories.append(relatedLinkFetch(x))
    
if ('Google' in titles) == True:
    ind = titles.index('Google')
    del(titles[ind])
    del(authors[ind])
    del(votes[ind])
    del(rel_stories[ind])

page_df = pd.DataFrame(
    {'Title': titles,
     'Author': authors,
     'Votes': votes,
     'Tags': tags,
     'Related Stories': rel_stories
    })

In [135]:
page_df.head()

,Title,Author,Votes,Tags,Related Stories
0,A Practical Guide to Evil,ErraticErrata,47572,"[adventure, anti-hero, coming-of-age, fantasy,...","[The Gods are Bastards, The Wandering Inn, Wor..."
1,Worm,Wildbow,39123,"[action, adventure, complete, crime, dark, hig...","[Pact, Twig, Ward, A Practical Guide to Evil, ..."
2,Ward,wildbow,24115,"[action, adventure, alternate-universe, crime,...","[Worm, Twig, Pact, A Practical Guide to Evil, ..."
3,The Wandering Inn,pirateaba,22305,"[fantasy, magic, ]","[A Practical Guide to Evil, The Gods are Basta..."
4,Metaworld Chronicles,Wutosama,20009,"[fantasy, magic, relationships, young-adult, ]","[A Practical Guide to Evil, The Wandering Inn,..."
